In [1]:
import bs4
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests 
import time

In [2]:
template = f'https://www.madlan.co.il/for-sale/?filters=_0-10000000_1-5_________0-100000_____&tracking_search_source=filter_apply&marketplace=residential'

In [3]:
def get_response(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)
    return response


In [4]:
#request for cruwling
def get_response_city(city, rooms_num):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    main_url = f'https://www.madlan.co.il/for-sale/{city}?filters=_0-10000000_1-{rooms_num}_________0-100000_____&tracking_search_source=filter_apply&marketplace=residential'
    
    response = requests.get(main_url, headers=headers)
    return response, main_url

In [5]:
def get_soup(response):
    soup_madlan = BeautifulSoup(response.content, 'html.parser')
    return soup_madlan



In [6]:
x=5
main_res,main_url=get_response_city('אשדוד-ישראל',x) #send name of city 

In [7]:
print(main_url) # that good so far

https://www.madlan.co.il/for-sale/אשדוד-ישראל?filters=_0-10000000_1-5_________0-100000_____&tracking_search_source=filter_apply&marketplace=residential


In [8]:
count_page= 7
soup_list =list()

def build_ds_for_test(response):
        soup=get_soup(response)
        soup_list.append(soup) # in postion zero --> page one
        count=1
        while count!=count_page:
            count +=1
            name_of_data_auto='bulletins-pagination-'+str(count) #create the full name to search  
            div_npage_name = soup.find('a', attrs={'data-auto': name_of_data_auto})
            if div_npage_name:
                print(count)
                temp_url='http://madlan.co.il' + div_npage_name['href']
                temp_res=get_response(temp_url)
                temp_soup=get_soup(temp_res)
                soup_list.append(temp_soup)
                soup=temp_soup
            else:
                print('No more pages')
                break
        return soup_list
        


In [9]:
currect_city='אשדוד'

In [10]:
all_pages_soup_listDB=build_ds_for_test(main_res)


2
3
4
5
6
7


In [11]:
len(all_pages_soup_listDB)
all_pages_soup_listDB

[<!DOCTYPE html>
 <style data-emotion-css="44lp2d">html,body,div,span,applet,object,iframe,h1,h2,h3,h4,h5,h6,p,blockquote,pre,a,abbr,acronym,address,big,cite,code,del,dfn,em,img,ins,kbd,q,s,samp,small,strike,strong,sub,sup,tt,var,b,u,i,center,dl,dt,dd,ol,ul,li,fieldset,form,label,legend,table,caption,tbody,tfoot,thead,tr,th,td,article,aside,canvas,details,embed,figure,figcaption,footer,header,hgroup,menu,nav,output,ruby,section,summary,time,mark,audio,video{margin:0;padding:0;border:0;font-size:100%;font:inherit;vertical-align:baseline;}article,aside,details,figcaption,figure,footer,header,hgroup,menu,nav,section{display:block;}body{line-height:1;overscroll-behavior:none;}ol,ul{list-style:none;}blockquote,q{quotes:none;}blockquote:before,blockquote:after,q:before,q:after{content:'';content:none;}table{border-collapse:collapse;border-spacing:0;}*{box-sizing:border-box;}body{font-family:novemberHebrew,Muli,sans-serif;}</style><style data-emotion-css="fyvm37">#overlay-root > div:not(:last

In [12]:
floor='6 חד׳'
print(len(floor))
print(floor[0])



5
6


In [20]:
def load_data_to_df(n_pages):
    ######################################################################################
    street_l = list()
    city_l = list()
    price_l = list()
    rooms_l = list()
    floor_l = list()
    floor_in_buildng_l= list()
    size_l = list()
    elvator_l = list()
    merger_l =list()
    shielding_l = list()
    storage_l = list()
    access_for_disabled_l = list()
    improved_l =list()
    parking_l=list()
    balcony_l =list()
    year_of_building= list()
    price_per_msqure_l=list()
    yield_l=list() # תשואה
    ######################################################################################
    
    ######################################################################################
    for index in range(0,6):
        for page in all_pages_soup_listDB:
                #######################################################################  fill the street list               
                street_div=page.find_all('h1', attrs ={'class': 'css-fgboif e10pmpfq4'})
                street_l.append(street_div)
                ####################################################################### fill the city list
                city_l.append(currect_city)
                ####################################################################### fill the price list
                price_div = page.find('div', attrs = {'class': 'css-1x3r91k e10pmpfq10'})
                if price_div:
                    price = price_div.text.strip().replace(',', '')  # remove comma and whitespace
                    price_l.append(price)
                ####################################################################### fill the rooms list + floor list + floors in building list + size list
                # get all elements with class 'example-class'
                example_class_elems = page.find_all(class_='css-ixartp e3vrfmg5')
                room_elem = example_class_elems[1].text.strip()
                rooms_l.append(room_elem[0]) #append to room list
                
                
                
                floor_elem = example_class_elems[2].text.strip() #append to floor and floors in bulidin list
                if len(floor_elem)==13: #when the num of floors<10
                    floor_l.append(floor_elem[5])
                    floor_in_buildng_l.append(floor_elem[12])
                elif len(floor_elem)==14: #when the num of floors >=10 but the currect floor is <10
                    floor_l.append(floor_elem[5])
                    subset_floors = floor_elem[12:14] 
                    floor_in_buildng_l.append(subset)
                elif len(floor_elem)==15: # when the num of floors>=10 and the currect floor is>=10
                    subset_floor = floor_elem[5:7]
                    floor_l.append(subset_floor)
                    subset_floors = floor_elem[12:15] 
                    floor_in_buildng_l.append(subset_floors)
                    
                
                size_elem=example_class_elems[3].text.strip() #append to size and floors in bulidin list
                if len(size_elem)==6:
                    subset_size= size_elem[0:2]
                    size_l.append(subset_size)
                elif len(size_elem)==7:
                    subset_size= size_elem[0:3]
                    size_l.append(subset_size)
                else:
                    size_l.append('No data on size')
                
                    
                ####################################################################### fill elvator + merger + parking + belcony + shilding + access + storage
                # get all elements with class 'example-class'
                block_of_data_onA = page.find_all(class_='css-stb5t9 elkstcv2')
                print(block_of_data_onA)
                elvator_elem = block_of_data_onA[0].text.strip()
                if elvator_elem=='אין מעלית':
                    elvator_l.append(0) #append 0 if no elvator 1 if have
                else:
                     elvator_l.append(1)
                        
                merge_elem = block_of_data_onA[1].text.strip()
                if elvator_elem=='יש מיזוג אויר':
                    elvator_l.append(1) #append 0 if no merger 1 if have
                else:
                     elvator_l.append(0)
                    
                parking_elem = block_of_data_onA[2].text.strip()
                if parking_elem=='אין חניה':
                    parking_l.append(0) #append 0 if no parking 1 if have
                else:
                     parking_l.append(1)
                        
                balcony_elem = block_of_data_onA[3].text.strip()
                if balcony_elem=='אין מרפסת':
                    balcony_l.append(0) #append 0 if no balcony 1 if have
                else:
                     balcony_l.append(1)
                
                shilding_elem = block_of_data_onA[5].text.strip()
                if shilding_elem=='אין ממ״ד':
                    shielding_l.append(0) #append 0 if no shilding room 1 if have
                else:
                     shielding_l.append(1)
                        
                storage_elem = block_of_data_onA[6].text.strip()
                if storage_elem=='אין מחסן':
                    storage_l.append(0) #append 0 if no storage 1 if have
                else:
                     storage_l.append(1)
                        
                access_for_disabled_elem = block_of_data_onA[7].text.strip()
                if access_for_disabled_elem=='לא נגיש לנכים':
                    access_for_disabled_l.append(0) #append 0 if no access 1 if have
                else:
                     access_for_disabled_l.append(1)
                        
                ####################################################################### fill improved list 
                improved_div=page.find('div', attrs ={'data-auto': 'unit-general-condition-value'}).text.strip()
                if improved_div == 'משופץ':
                    improved_l.append(1)
                else:
                    improved_l.append(0)
                   
                
                        
                

        
    df=pd.DataFrame({'city':city_l ,'street':street_l,'price':price_l ,'rooms':rooms_l , 
                    'floor':floor_l ,'floors_in_building':floor_in_buildng_l,'size':size_l ,'elvator':elvator_l ,
                    'merger':merger_l ,'shilding_room':shielding_l,'access_for_disabled':access_for_disabled_l ,'improved':improved_l , })
    return df

In [21]:
main_df= load_data_to_df(6)

[]


IndexError: list index out of range

In [15]:


while count != 3:
        index_x=count
        cuur_url=next_page[len(next_page)-1]
        res=get_response(cuur_url)
        print(res)
        soup = get_soup(res) # in the begginnig that page 1
        links_for_apartment=soup.find_all('a', attrs= {'data-auto':'listed-bulletin-clickable'}) 
        if links_for_apartment:
            for link in links_for_apartment:
                all_apartment_links.append('madlan.co.il'+link['href'])
                
                      
       #name_of_data_auto='bulletins-pagination-'+str(count) #create the full name to search  
        #div_npage_name = soup.find('a', attrs={'data-auto': name_of_data_auto})
        #if div_npage_name:
        #    next_page.append('http://madlan.co.il' + div_npage_name['href'])
        #else:
         #   print('No more pages')
          #  break
        
        count +=1
       
        

NameError: name 'count' is not defined

In [19]:
list_of_alinks= create_list_of_all_links(10,main_url)
print(list_of_alinks)
street_l = list()
for link in list_of_alinks:
        res=get_response(link)
        soup = get_soup(res) 
        last_div=soup.find('h1', {'data-auto': 'primary_address_text'}) 
        street_l.append(last_div.text.strip())
    
list_of_alinks

['madlan.co.il/listings/jysQtmZsSDw', 'madlan.co.il/listings/Se50P3iXZ58', 'madlan.co.il/projects/%D7%9B%D7%95%D7%9B%D7%91_%D7%94%D7%A6%D7%A4%D7%95%D7%9F_%D7%90%D7%A9%D7%93%D7%95%D7%93?term=%D7%90%D7%A9%D7%93%D7%95%D7%93-%D7%99%D7%A9%D7%A8%D7%90%D7%9C&filters=_0-10000000_1-5_________0-100000_____&tracking_search_source=filter_apply&tracking_event_source=list_regular_card&tracking_list_index=2&dealType=sale', 'madlan.co.il/projects/%D7%94%D7%A8%D7%91_%D7%A9%D7%90%D7%95%D7%9C%D7%99_21_%D7%90%D7%A9%D7%93%D7%95%D7%93?term=%D7%90%D7%A9%D7%93%D7%95%D7%93-%D7%99%D7%A9%D7%A8%D7%90%D7%9C&filters=_0-10000000_1-5_________0-100000_____&tracking_search_source=filter_apply&tracking_event_source=list_regular_card&tracking_list_index=3&dealType=sale', 'madlan.co.il/listings/cewgG3R8Kqm', 'madlan.co.il/listings/doj6TlTEMRe', 'madlan.co.il/listings/fklXM7904e9', 'madlan.co.il/listings/hfH4NlX0QqE', 'madlan.co.il/projects/Central_Park_17?term=%D7%90%D7%A9%D7%93%D7%95%D7%93-%D7%99%D7%A9%D7%A8%D7%90%D7%9C&

MissingSchema: Invalid URL 'madlan.co.il/listings/jysQtmZsSDw': No scheme supplied. Perhaps you meant https://madlan.co.il/listings/jysQtmZsSDw?

In [18]:

soup_madlan=get_soup(main_res)
madlan_df= load_data_to_df(soup_madlan,main_url)

TypeError: load_data_to_df() takes 1 positional argument but 2 were given

In [17]:
def create_list_of_all_links(num_of_page,main_url): # one page = 50 result
    page_name_place =None
    next_page=list()
    next_page.append(main_url)

    count =1

    all_apartment_links=list() #create list with link to all 

    while count != num_of_page:
        index_x=count
        cuur_url=next_page[len(next_page)-1]
        res=get_response(cuur_url)
        soup = get_soup(res) # in the begginnig that page 1
        
        links_for_apartment=soup.find_all('a', attrs= {'data-auto': 'listed-bulletin-clickable'})  
        if links_for_apartment:
            for link in links_for_apartment:
                all_apartment_links.append('madlan.co.il'+link['href'])
                
                
                
                    
        name_of_data_auto='bulletins-pagination-'+str(count) #create the full name to search  
        div_npage_name = soup.find('a', attrs={'data-auto': name_of_data_auto})
        if div_npage_name:
            next_page.append('http://madlan.co.il' + div_npage_name['href'])
        else:
            print('No more pages')
            break
        
        count +=1
    return all_apartment_links
